# 라이브러리 및 데이터

In [1]:
import random
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix
import joblib
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#시드 고정
DEFAULT_RANDOM_SEED = 3031
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

test = pd.read_csv('test.csv')
test

,X1,X3,X5,X6,X7a,X7b,X7c,X7d,X7e,X7f,...,X71_binary,X72_binary,X73_binary,X74_binary,X75_binary,X76_binary,X77_binary,X78_binary,anomaly,score
0,40766,3.772000e+03,0.0,0.0,0.0,0.0,0.0,0.0,158424.0,416172.0,...,0,0,0,0,0,0,0,0,1,0.096545
1,9030,1.740000e+02,0.0,0.0,0.0,0.0,0.0,0.0,600.0,136934.0,...,0,0,0,0,0,0,0,0,1,0.068260
2,93156,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,137692.0,2382150.0,...,1,1,1,1,1,1,1,1,-1,-0.042344
3,29084,2.940000e+02,0.0,0.0,0.0,0.0,0.0,7032.0,275686.0,942348.0,...,0,0,0,0,0,0,0,0,1,0.122501
4,724,4.200000e+01,0.0,0.0,0.0,0.0,3276.0,4980.0,16332.0,44564.0,...,0,0,0,0,0,0,0,0,1,0.132192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,8,2.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,106.0,3500.0,...,0,0,0,0,0,0,0,0,1,0.135068
20996,43794,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,348.0,133788.0,...,0,0,0,0,0,0,0,0,1,0.114759
20997,80526,1.900000e+02,0.0,0.0,0.0,0.0,0.0,192.0,545330.0,2484434.0,...,0,0,0,0,0,0,0,0,1,0.031726
20998,80214,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,7914.0,774048.0,...,0,0,0,0,0,0,0,0,1,0.077485


# XGB Classifier_튜닝X

In [2]:
seedBasic(42)
model = joblib.load('XGBbasic.pkl')

In [3]:
%%time
probabilities = model.predict_proba(test)[:, 1]

# 새로운 threshold 설정
new_threshold = 0.0015

# threshold를 기준으로 예측 레이블 생성
new_predictions = (probabilities >= new_threshold).astype(int)

CPU times: total: 750 ms
Wall time: 346 ms


In [4]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = new_predictions
sub.to_csv('xgb_bin.csv', index=False)

# XGB Classifier_best튜닝

In [5]:
seedBasic(42)
model = joblib.load('XGBbest.pkl')

In [6]:
%%time
probabilities = model.predict_proba(test)[:, 1]

# 새로운 threshold 설정 
new_threshold = 0.002

# threshold를 기준으로 예측 레이블 생성
new_predictions = (probabilities >= new_threshold).astype(int)

CPU times: total: 578 ms
Wall time: 221 ms


In [7]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = new_predictions
sub.to_csv('xgb_best.csv', index=False)

# LGBM Regressor_튜닝X

In [8]:
seedBasic(42)
model = joblib.load('lgbrbasic.pkl')

In [9]:
%%time
probabilities = model.predict_proba(test)[:, 1]

# 새로운 threshold 설정 
new_threshold = 0.038

# threshold를 기준으로 예측 레이블 생성
new_predictions = (probabilities >= new_threshold).astype(int)

CPU times: total: 734 ms
Wall time: 279 ms


In [10]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = new_predictions
sub.to_csv('lgbm_bin.csv', index=False)

# LGBM Regressor_best튜닝

In [11]:
seedBasic(42)
model = joblib.load('lgbrbest.pkl')

In [12]:
%%time
probabilities = model.predict_proba(test)[:, 1]

# 새로운 threshold 설정 
new_threshold = 0.038

# threshold를 기준으로 예측 레이블 생성
new_predictions = (probabilities >= new_threshold).astype(int)

CPU times: total: 641 ms
Wall time: 301 ms


In [13]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = new_predictions
sub.to_csv('lgbm_best.csv', index=False)

# CatBoost

In [14]:
seedBasic(42)
model = joblib.load('cat.pkl')

In [15]:
%%time
probabilities = model.predict_proba(test)[:, 1]

# 새로운 threshold 설정 
new_threshold = 0.038

# threshold를 기준으로 예측 레이블 생성
new_predictions = (probabilities >= new_threshold).astype(int)

CPU times: total: 391 ms
Wall time: 528 ms


In [16]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = new_predictions
sub.to_csv('catboost.csv', index=False)

# Voting
### 최종 선정: 1. XGB 튜닝X / 2. XGB 튜닝 / 3. LGBM Regressor 튜닝X

In [17]:
sub1 = pd.read_csv('xgb_bin.csv') # XGBClassifier Bin
sub2 = pd.read_csv('xgb_best.csv') # XBClassifier Best
sub3 = pd.read_csv('lgbm_bin.csv') # LGBMRegressor 깡통

print(sum(sub1['target'] ==1))
print(sum(sub2['target'] ==1))
print(sum(sub3['target'] ==1))

1216
1174
678


In [18]:
concat = pd.concat([sub1, sub2, sub3],axis=1)
concat = concat[['target']]
concat.columns = [1,2,3]

In [19]:
concat['sum'] = (concat[1] + concat[2] + concat[3])

for i in range(3):
  print(sum(concat['sum'] == i))

19633
340
353


In [20]:
# 3 모델 중 2개 이상만 class=1이면 전부 1
concat['class_pred'] = [0 for i in range(21000)]

for i in range(21000):
  if concat['sum'][i] >= 2:
    concat['class_pred'][i] = 1

sum(concat['class_pred']==1)

1027

In [21]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = concat['class_pred']
sub.to_csv('final_sub.csv', index=False)